In [1]:
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.scoring_helpers import weighted_sum
from src.config import parameters, thresholds


In [2]:
sri_df = pd.read_csv(
    PROJECT_ROOT / "data" / "processed" / "sri_output.csv"
)

cbr_df = pd.read_csv(
    PROJECT_ROOT / "data" / "processed" / "cbr_output.csv"
)

print(sri_df.head())
print(cbr_df.head())


    region_id    month  sri_score sri_category
0  Puducherry  2025-09  -0.696452       Stable
1  Puducherry  2025-09  -0.272371       Stable
2  Puducherry  2025-09  -0.342865       Stable
3  Puducherry  2025-09  -0.534098       Stable
4  Puducherry  2025-09   1.060471      Fragile
    region_id    month  cbr_score risk_band
0  Puducherry  2025-09   -0.32333       Low
1  Puducherry  2025-09   -0.32333       Low
2  Puducherry  2025-09   -0.32333       Low
3  Puducherry  2025-09   -0.32333       Low
4  Puducherry  2025-09   -0.32333       Low


In [3]:
sri_monthly = (
    sri_df
    .groupby(["region_id", "month"], as_index=False)
    .agg({"sri_score": "mean"})
)

cbr_monthly = (
    cbr_df
    .groupby(["region_id", "month"], as_index=False)
    .agg({"cbr_score": "mean"})
)

assert sri_monthly.duplicated(["region_id", "month"]).sum() == 0
assert cbr_monthly.duplicated(["region_id", "month"]).sum() == 0


In [4]:
ipr_df = pd.merge(
    sri_monthly,
    cbr_monthly,
    on=["region_id", "month"],
    how="inner",
    validate="one_to_one"
)


In [5]:
ipr_df["priority_score"] = weighted_sum(
    ipr_df[["sri_score", "cbr_score"]],
    weights=parameters["IPR_WEIGHTS"]
)


In [6]:
IPR_THRESHOLDS = thresholds["IPR"]

ESCALATE = IPR_THRESHOLDS["ESCALATE"]
INTERVENE = IPR_THRESHOLDS["INTERVENE"]

def classify(score):
    if score >= ESCALATE:
        return "Escalate"
    elif score >= INTERVENE:
        return "Intervene"
    else:
        return "Monitor"

ipr_df["recommended_action"] = ipr_df["priority_score"].apply(classify)

ipr_df.head()


,region_id,month,sri_score,cbr_score,priority_score,recommended_action
0,Puducherry,2025-06,-0.460436,0.128583,-0.165927,Monitor
1,Puducherry,2025-09,0.346486,-0.051769,0.147359,Monitor
2,Puducherry,2025-10,-0.097142,-0.095068,-0.096105,Monitor
3,Puducherry,2025-11,-0.003532,0.006129,0.001298,Monitor
4,Puducherry,2025-12,-0.184848,0.016315,-0.084267,Monitor


In [8]:
ipr_df.to_csv(
    PROJECT_ROOT / "data" / "processed" / "ipr_output.csv",
    index=False
)
